## Predicting Survival on the Titanic

### History
Perhaps one of the most infamous shipwrecks in history, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 people on board. Interestingly, by analysing the probability of survival based on few attributes like gender, age, and social status, we can make very accurate predictions on which passengers would survive. Some groups of people were more likely to survive than others, such as women, children, and the upper-class. Therefore, we can learn about the society priorities and privileges at the time.

### Assignment:

Build a Machine Learning Pipeline, to engineer the features in the data set and predict who is more likely to Survive the catastrophe.

Follow the Jupyter notebook below, and complete the missing bits of code, to achieve each one of the pipeline steps.

In [ ]:
import re

# to handle datasets
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import StandardScaler

# to build the models
from sklearn.linear_model import LogisticRegression

# to evaluate the models
from sklearn.metrics import accuracy_score, roc_auc_score

# to persist the model and the scaler
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

## Prepare the data set

In [ ]:
# load the data - it is available open source and online

data = pd.read_csv('https://www.openml.org/data/get_csv/16826755/phpMYEkMl')

# display data
data.head()

In [ ]:
# replace interrogation marks by NaN values

data = data.replace('?', np.nan)

In [ ]:
# retain only the first cabin if more than
# 1 are available per passenger

def get_first_cabin(row):
    try:
        return row.split()[0]
    except:
        return np.nan
    
data['cabin'] = data['cabin'].apply(get_first_cabin)

In [ ]:
# extracts the title (Mr, Ms, etc) from the name variable

def get_title(passenger):
    line = passenger
    if re.search('Mrs', line):
        return 'Mrs'
    elif re.search('Mr', line):
        return 'Mr'
    elif re.search('Miss', line):
        return 'Miss'
    elif re.search('Master', line):
        return 'Master'
    else:
        return 'Other'
    
data['title'] = data['name'].apply(get_title)

In [ ]:
# cast numerical variables as floats

data['fare'] = data['fare'].astype('float')
data['age'] = data['age'].astype('float')

In [ ]:
# drop unnecessary variables

data.drop(labels=['name','ticket', 'boat', 'body','home.dest'], axis=1, inplace=True)

# display data
data.head()

In [ ]:
print(data.shape)

In [ ]:
# save the data set
data.to_csv('titanic.csv', index=False)

## Data Exploration

### Find numerical and categorical variables

In [ ]:
target = 'survived'

In [ ]:
vars_num = [var for var in data.columns if data[var].dtype != 'O' and var != 'survived']
vars_cat = [var for var in data.columns if data[var].dtype == 'O']

print('Number of numerical variables: {}'.format(len(vars_num)))
print('Number of categorical variables: {}'.format(len(vars_cat)))

In [ ]:
print(vars_num)
print(vars_cat)

### Find missing values in variables

In [ ]:
# make a list of the variables that contain missing values
vars_with_na = [var for var in data.columns if data[var].isnull().sum() > 0]

# determine percentage of missing values (expressed as decimals)
# and display the result ordered by % of missing data
data[vars_with_na].isnull().mean().sort_values(ascending=False)

In [ ]:
# first in numerical variables
num_na = [var for var in vars_num if var in vars_with_na]
print(num_na)

In [ ]:
# now in categorical variables
cat_na = [var for var in vars_cat if var in vars_with_na]
print(cat_na)

### Determine cardinality of categorical variables

In [ ]:
data[vars_cat].nunique().sort_values(ascending=False).plot.bar(figsize=(12,5))

### Determine the distribution of numerical variables

In [ ]:
data[vars_num].hist(bins=30, figsize=(15,15))
plt.show()

## Separate data into train and test

Use the code below for reproducibility. Don't change it.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('survived', axis=1),  # predictors
    data['survived'],  # target
    test_size=0.2,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train.shape, y_test.shape

## Feature Engineering

### Extract only the letter (and drop the number) from the variable Cabin

In [ ]:
X_train['cabin'] = X_train['cabin'].str[0]
X_test['cabin'] = X_test['cabin'].str[0]

In [ ]:
# check
print(X_train.shape)
print(X_train['cabin'])

### Fill in Missing data in numerical variables:

- Add a binary missing indicator
- Fill NA in original variable with the median

In [ ]:
for var in num_na:
    # calculate the median using the train set
    median_val = X_train[var].median()
    print(var, median_val)

    # add binary missing indicator (in train and test)
    X_train[var + '_na'] = np.where(X_train[var].isnull(), 1, 0)
    X_test[var + '_na'] = np.where(X_test[var].isnull(), 1, 0)

    # replace missing values by the mean
    # (in train and test)
    X_train[var].fillna(median_val, inplace=True)
    X_test[var].fillna(median_val, inplace=True)

# check that we have no more missing values in the engineered variables
X_train[num_na].isnull().sum()
X_test[num_na].isnull().sum()

### Replace Missing data in categorical variables with the string **Missing**

In [ ]:
for var in cat_na:
    X_train[cat_na] = X_train[cat_na].fillna('Missing')
    X_test[cat_na] = X_test[cat_na].fillna('Missing')

In [ ]:
# check
print(X_train[cat_na])
print(X_test[cat_na])

### Remove rare labels in categorical variables

- remove labels present in less than 5 % of the passengers

In [ ]:
def find_less_frequent_labels(df, var, rare_perc):
    
    # function finds the labels that are shared by less than
    # a certain % of the rows in the dataset
    # returns pandas.Index type

    df = df.copy()

    tmp = df.groupby(var)[var].count() / len(df)

    return tmp[tmp < rare_perc].index

In [ ]:
for var in vars_cat:
    # find the less_frequent labels
    less_frequent_ls = find_less_frequent_labels(X_train, var, 0.05)
    
    print(var, less_frequent_ls)
    print()
    train_mask = X_train[var].isin(less_frequent_ls)
    test_mask = X_test[var].isin(less_frequent_ls)
    # keep only frequent labels
    X_train = X_train[train_mask == False]
    X_test = X_test[test_mask == False]
    
    # remove corresponding target rows
    y_train = y_train[train_mask == False]
    y_test = y_test[test_mask == False]

### Perform one hot encoding of categorical variables into k-1 binary variables

- k-1, means that if the variable contains 9 different categories, we create 8 different binary variables
- Remember to drop the original categorical variable (the one with the strings) after the encoding

In [ ]:
for var in vars_cat:
    unique_labels = X_train[var].unique()
    one_hot_labels = np.arange(X_train[var].nunique())
    mapping = dict(zip(unique_labels, one_hot_labels))
    
    X_train[var] = X_train[var].map(mapping)
    X_train[var] = X_train[var].astype('int64')
    
    unique_labels = X_test[var].unique()
    one_hot_labels = np.arange(X_test[var].nunique())
    mapping = dict(zip(unique_labels, one_hot_labels))
    
    X_test[var] = X_test[var].map(mapping)
    X_test[var] = X_test[var].astype('int64')

### Scale the variables

- Use the standard scaler from Scikit-learn

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(
    scaler.transform(X_train),
    columns=X_train.columns
)

X_test = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_train.columns
)

## Train the Logistic Regression model

- Set the regularization parameter to 0.0005
- Set the seed to 0

In [ ]:
log_reg_model = LogisticRegression(C=2000.0, random_state=0)
log_reg_model.fit(X_train, y_train)

## Make predictions and evaluate model performance

Determine:
- roc-auc
- accuracy

**Important, remember that to determine the accuracy, you need the outcome 0, 1, referring to survived or not. But to determine the roc-auc you need the probability of survival.**

In [ ]:
# training dataset
pred = log_reg_model.predict(X_train)
pred_prob = log_reg_model.predict_proba(X_train)
print('train accuracy: ', accuracy_score(y_train, pred))
print('train roc-auc score ', roc_auc_score(y_train, pred_prob[:, 1]))

# testing dataset
pred = log_reg_model.predict(X_test)
pred_prob = log_reg_model.predict_proba(X_test)
print('test accuracy: ', accuracy_score(y_test, pred))
print('test roc-auc score ', roc_auc_score(y_test, pred_prob[:, 1]))


That's it! Well done

**Keep this code safe, as we will use this notebook later on, to build production code, in our next assignement!!**